Large Steps in Inverse Rendering of Geometry
======================================

This is an example on how to use our method for shape optimization with differentiable rendering. It uses `nvdiffrast` for the differentiable rendering part.

In [ ]:
import torch
from tqdm import trange
import os

# 0. Loading the scene

Let's start by loading a scene

In [ ]:
from scripts.load_xml import load_scene
import os

In [ ]:
# Load the scene
filepath = os.path.join(os.getcwd(), "scenes", "nefertiti", "nefertiti.xml")
scene_params = load_scene(filepath)

# Load reference shape
v_ref = scene_params["mesh-target"]["vertices"]
n_ref = scene_params["mesh-target"]["normals"]
f_ref = scene_params["mesh-target"]["faces"]

# Load source shape
v = scene_params["mesh-source"]["vertices"]
f = scene_params["mesh-source"]["faces"]

# 1. Rendering references

Then, we need to setup a differentiable rendering pipeline. Here, we use an implementation based on `nvdiffrast`:

In [ ]:
from scripts.render import NVDRenderer

We initialize it once, so it loads the camera data, the environment map and precomputes the shading model, using spherical harmonics in this case.

In [ ]:
# Initialize the renderer
renderer = NVDRenderer(scene_params, shading=True, boost=3)

Let's render the target shape to use as a reference for the optimization:

In [ ]:
ref_imgs = renderer.render(v_ref, n_ref, f_ref)

Let's look at one of these references

In [ ]:
import matplotlib.pyplot as plt
plt.imshow((ref_imgs[6,...,:-1].clip(0,1).pow(1/2.2)).cpu().numpy(), origin='lower')

# 2. Parameterizing

Now it's time to setup the optimization. First, let us import what we need. We need an optimizer, `AdamUniform`, and the functions that allow us to convert back and forth between vertex positions and their parameterization.

In [ ]:
from largesteps.parameterize import from_differential, to_differential
from largesteps.geometry import compute_matrix

In [ ]:
steps = 2000 # Number of optimization steps
step_size = 1e-1 # Step size
lambda_ = 49 # Hyperparameter lambda of our method, used to compute the matrix (I + lambda_*L)

Now we need to parameterize our shape. 

In [ ]:
# Compute the system matrix
M = compute_matrix(v, f, lambda_)

# Parameterize
u = to_differential(M, v)

Let's also optimize a translation of the shape at the same time

In [ ]:
# Initialize the optimized variables and the optimizer
tr = torch.zeros((1,3), device='cuda', dtype=torch.float32)

Let's initialize our optimizer, `AdamUniform`

In [ ]:
from largesteps.optimize import AdamUniform

In [ ]:
tr.requires_grad = True
u.requires_grad = True
opt_params = [
    # The results in the paper were generated using a slightly different
    # implementation of the system matrix than this one, so we need to
    # scale the step size by this factor to match the results exactly.
    {'params': tr, 'lr': step_size / (1 + lambda_)},
    {'params': u}
]
opt = AdamUniform(opt_params, step_size)

In [ ]:
# Dictionary that is returned in the end, contains useful information for debug/analysis
v_steps = torch.zeros((steps+1, *v.shape), device='cuda')
losses = torch.zeros(steps+1, device='cuda')

And now we can run our optimization. The only difference with "regular" optimization here is the call to `from_differential` in the loop body, that converts the parameterization to vertex coordinates. The rest of the optimization pipeline is unchanged.

In [ ]:
from scripts.geometry import compute_vertex_normals, compute_face_normals
# Optimization loop
for it in trange(steps):

    # Get cartesian coordinates for parameterization
    v = from_differential(M, u, 'Cholesky')

    # Recompute vertex normals
    face_normals = compute_face_normals(v, f)
    n = compute_vertex_normals(v, f, face_normals)

    # Render images
    opt_imgs = renderer.render(tr + v, n, f)

    # Compute L1 image loss
    loss = (opt_imgs - ref_imgs).abs().mean()

    # Record optimization state for later processing
    with torch.no_grad():
        losses[it] = loss
        v_steps[it] = v + tr

    # Backpropagate
    opt.zero_grad()
    loss.backward()
    
    # Update parameters
    opt.step() 

In [ ]:
with torch.no_grad():
    # Render images
    opt_imgs = renderer.render(tr + v, n, f)
    # Compute L1 image loss
    loss = (opt_imgs - ref_imgs).abs().mean()
    losses[-1] = loss
    v_steps[-1] = v + tr

In [ ]:
from meshplot import plot
from ipywidgets import interact
import numpy as np

In [ ]:
v_numpy = v_steps.cpu().numpy()
f_numpy = f.cpu().numpy()

In [ ]:
shading_params = {
    "width": 600, "height": 600,
    "antialias": True,
    "colormap": "viridis",
    "wireframe": True, "wire_width": 0.03, "wire_color": "black"
}

In [ ]:
@interact(it=(0, steps-1))
def plot_verts(it):
    plot(v_numpy[it], f_numpy, shading=shading_params)